In [139]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import glob
import json
from pathlib import Path

In [140]:
def load_json(dir):
    try:
        with open(dir) as f:
            data = json.load(f)
            
        return data
    
    except:
        return None

In [141]:
RESULT_DIR = "..\\result_summary"

In [145]:
def parse_file_nm(file_nm):
    mcts_info = None
    
    components = file_nm.split("\\")
    
    if "debug" in components:
        components.remove("debug")
    
    method = components[2]
    
    diff_cut = 1
    
    if 'am' in method:
        _, _, method, _, prob_type, num_probs, model_info, baseline_info, _ = components
        
    elif 'mcts' in method:
        _, _, method, diff_cut, _, prob_type, num_probs, model_info, baseline_info, mcts_info, _ = components
        diff_cut = float(diff_cut.split("-")[-1])

    num_probs = num_probs.split("-")[0]
    num_probs = int(num_probs.split("_")[1])
    baseline_info = baseline_info.split("-")[-1]
    
    return method, prob_type, num_probs, model_info, baseline_info, mcts_info, diff_cut

In [146]:
for file in glob.glob(f"{RESULT_DIR}/**/*.json", recursive=True):
    result = load_json(file)

    avg_result = False
    for k, v in result.items():
        for k2, v2 in v.items():
            if 'result' in k2:
                avg_result = True
                break
    
    if not avg_result and len(result) != 102:
        print(len(result))
        print(file)
        
# need to do diff-0.75 and CVRP with N=50 for 4 and 6

In [182]:
# parse all json file named with "all_result_avg.json" from the RESULT_DIR

all_result = {}

for file in glob.glob(f"{RESULT_DIR}/**/all_result_avg.json", recursive=True):
    data = load_json(file)
    # print(file)
    method, prob_type, num_probs, model_info, baseline_info, mcts_info, diff_cut = parse_file_nm(file)
    
    if data is not None:
        
        if method not in all_result:
            all_result[method] = {}
        
        if prob_type not in all_result[method]:
            all_result[method][prob_type] = {}
            
        if num_probs not in all_result[method][prob_type]:
            all_result[method][prob_type][num_probs] = {}
        
        if mcts_info is None:
            if model_info not in all_result[method][prob_type][num_probs]:
                all_result[method][prob_type][num_probs][model_info] = {}
            
            if baseline_info not in all_result[method][prob_type][num_probs][model_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info] = data
                
        else:
            if model_info not in all_result[method][prob_type][num_probs]:
                all_result[method][prob_type][num_probs][model_info] = {}
                
            if baseline_info not in all_result[method][prob_type][num_probs][model_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info] = {}
                
            if mcts_info not in all_result[method][prob_type][num_probs][model_info][baseline_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info][mcts_info] = {}
                
            if diff_cut not in all_result[method][prob_type][num_probs][model_info][baseline_info][mcts_info]:
                all_result[method][prob_type][num_probs][model_info][baseline_info][mcts_info][diff_cut] = data
                
    # break

In [183]:
def get_parital_result(method, prob_type, num_prob):       
    partial_result = {'model_info': [], 'baseline': [], 'mcts_info': [], 'epoch': [], 'score': [], 'score_std': [], 'runtime': [],  'runtime_std': [], 'diff_cut': []}
    
    def _append_epoch_result(model_info, baseline_info, epoch, mcts, epoch_score, epoch_score_std, epoch_runtime, epoch_runtime_std, diff_cut):
        partial_result['model_info'].append(model_info)
        partial_result['baseline'].append(baseline_info)
        partial_result['mcts_info'].append(mcts)
        partial_result['epoch'].append(epoch)                
        partial_result['score'].append(epoch_score)
        partial_result['score_std'].append(epoch_score_std)
        partial_result['runtime'].append(epoch_runtime)
        partial_result['runtime_std'].append(epoch_runtime_std)
        partial_result['diff_cut'].append(diff_cut)

    target_result = all_result[method][prob_type][num_prob]
    
    if "am" in method:
        for model_info in target_result:
            for baseline_info in target_result[model_info]:
                all_epoch_results = target_result[model_info][baseline_info]
                
                for epoch in all_epoch_results:
                    current_epoch_result = all_epoch_results[epoch]['result_avg']
                    epoch_score = current_epoch_result['score']
                    epoch_score_std = all_epoch_results[epoch]['result_std']['score']
                    epoch_runtime = current_epoch_result['runtime']
                    epoch_runtime_std = all_epoch_results[epoch]['result_std']['runtime']
                    
                    _append_epoch_result(model_info, baseline_info, epoch, "am", epoch_score,epoch_score_std, epoch_runtime, epoch_runtime_std, 1)
                    
    elif 'mcts' in method:
        for model_info in target_result:
            for baseline_info in target_result[model_info]:
                all_mcts_results = target_result[model_info][baseline_info]
                
                for mcts_info in all_mcts_results:
                    all_diff_results = all_mcts_results[mcts_info]
                    
                    for diff_cut in all_diff_results:
                        all_epoch_results = all_diff_results[diff_cut]
                    
                        for epoch in all_epoch_results:
                            current_epoch_result = all_epoch_results[epoch]['result_avg']
                            epoch_score = current_epoch_result['score']
                            epoch_score_std = all_epoch_results[epoch]['result_std']['score']
                            epoch_runtime = current_epoch_result['runtime']
                            epoch_runtime_std = all_epoch_results[epoch]['result_std']['runtime']
                            
                            _append_epoch_result(model_info, baseline_info, epoch, mcts_info, epoch_score,epoch_score_std, epoch_runtime, epoch_runtime_std, diff_cut)                                         
            
    df = pd.DataFrame.from_dict(partial_result)
    df['activation'] = df['model_info'].apply(lambda x: x.split("-")[5])
    df['enc_layers'] = df['model_info'].apply(lambda x: x.split('-')[2])
    df = df.drop(columns=['model_info'])
    
    df['train_score'] = df.apply(lambda x: x['epoch'].split("-")[1].split("=")[1], axis=1).astype(float)
    df['epoch'] =  df.apply(lambda x: x['epoch'].split("-")[0].split("=")[1], axis=1).astype(int)
    
    if 'mcts' in method:
        df['cpuct'] = df['mcts_info'].apply(lambda x: x.split("-")[2].split('_')[1])
        df['ns'] = df['mcts_info'].apply(lambda x: x.split("-")[0].split('_')[1])
        df['cpuct'] = df['cpuct'].astype(float)
        df['ns'] = df['ns'].astype(int)
    
    elif 'am' in method:
        df['cpuct'] = 0
        df['ns'] = 0
        
    df = df.drop(columns=['mcts_info'])
    

        
    return df

In [184]:
def plot_bar_result(base_df, baseline, activation, prob_type, num_prob, plot_dev=False, hue='cpuct'):
    if prob_type == 'tsp':
        y_ranges = {20: (3.75, 3.95), 50: (5.7, 5.875), 100: (7.95, 8.25)}
        
    elif prob_type == 'cvrp':
        y_ranges = {20: (6.1, 7.5), 50: (9, 12), 100: (7.95, 8.25)}
        
    _df = base_df[(base_df['baseline'] == baseline) & (base_df['activation'] == activation)]
    
    # drop rows where the mcts_info is 0.8
    _df = _df[_df['cpuct'] != '0.8']
    
    # sort by the train score
    _df = _df.sort_values(by=['epoch', 'cpuct', 'ns']).reset_index(drop=True)
    _df['score_std'] = _df['score_std'].astype(float)
    
    _df['cpuct'] = _df['cpuct'].astype(object)
    _df['cpuct'] = _df['cpuct'].fillna('am')
    
    _df['ns'] = _df['ns'].astype(object)
    _df['ns'] = _df['ns'].fillna('am')
    
    plt.figure(figsize=(12, 7))
    
    ax = sns.barplot(data=_df, x=_df['epoch'], y='score', hue=hue)
    
    if plot_dev:
        bar_coords = [(rect.get_x(), rect.get_y(), rect.get_width(), rect.get_height()) for rect in ax.patches]
                
        for i, (x, y, w, h) in enumerate(bar_coords):
            x_pos = x + w / 2
            y_top = y + h + 0.01
            
            plt.errorbar(x=x_pos, y=y_top, yerr=_df['score_std'][i], fmt='none', color='black', capsize=4)
    
    plt.title(f"{prob_type}_{num_prob}_{baseline}_{activation}")
    plt.ylim(*y_ranges[num_prob])
    plt.legend()
    
    path = Path(f"../result_image/bars")
    
    if not path.exists():
        path.mkdir(parents=True)
        
    
    plt.savefig(f"{path}/{prob_type}_{num_prob}_{baseline}_{activation}.png")
    plt.show()

In [185]:
# tsp_20 = pd.concat([get_parital_result('am', 'tsp', 20), get_parital_result('mcts_v2', 'tsp', 20)])

# for _baseline in ['mean', 'val']:
#     for _activation in ['relu', 'swiglu']:
#         plot_bar_result(tsp_20, _baseline, _activation, 'tsp', 20, hue='cpuct')

In [186]:
tsp_100 = pd.concat([get_parital_result('am', 'tsp', 100), get_parital_result('mcts', 'tsp', 100)])
tsp_100.to_csv("../result_csv/tsp_100.csv")
tsp_100.sort_values(by=['score']).head(15)

,baseline,epoch,score,score_std,runtime,runtime_std,diff_cut,activation,enc_layers,train_score,cpuct,ns
499,val,299,7.952167,0.072568,10.248525,25.766765,0.75,swiglu,6,7.86777,1.1,1000
414,mean,299,7.952278,0.073115,1.087797,0.199681,0.75,swiglu,6,7.90623,1.1,100
527,val,299,7.952654,0.072702,0.581626,0.353638,0.50,swiglu,6,7.86777,1.1,500
497,val,299,7.952794,0.072653,0.929683,1.953379,0.50,swiglu,6,7.86777,1.1,1000
464,mean,299,7.952995,0.073776,4.318947,5.818327,0.75,swiglu,6,7.90623,1.1,500
514,val,299,7.953113,0.072455,0.438101,0.087440,0.50,swiglu,6,7.86777,1.1,250
531,val,249,7.953231,0.071119,5.765433,5.832914,0.75,swiglu,6,7.84949,1.1,500
450,mean,299,7.953557,0.073392,3.438926,2.169177,0.75,swiglu,6,7.90623,1.1,250
483,val,299,7.953631,0.074261,1.265813,0.224949,0.75,swiglu,6,7.86777,1.1,100
433,mean,299,7.953920,0.072868,8.395795,24.720814,0.75,swiglu,6,7.90623,1.1,1000


In [187]:
for env_type in ['tsp', 'cvrp']:
    for N in [20, 50, 100]:
        df = pd.concat([get_parital_result('am', env_type, N), get_parital_result('mcts', env_type, N)])
        df.to_csv(f"../result_csv/{env_type}_{N}.csv", index=False)

In [188]:
df

,baseline,epoch,score,score_std,runtime,runtime_std,diff_cut,activation,enc_layers,train_score,cpuct,ns
0,mean,49,16.717783,3.231875,0.471543,0.000779,1.00,relu,4,16.39185,0.0,0
1,mean,99,16.606976,3.071906,0.473988,0.001013,1.00,relu,4,16.41725,0.0,0
2,mean,149,16.574835,2.968623,0.479116,0.001992,1.00,relu,4,16.01232,0.0,0
3,mean,199,16.535594,3.113413,0.480392,0.001704,1.00,relu,4,16.35231,0.0,0
4,mean,249,16.490532,3.008314,0.476007,0.000952,1.00,relu,4,15.99893,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
346,val,199,16.474388,3.018032,11.897707,14.392978,0.75,swiglu,6,16.17082,1.1,500
347,val,149,16.473369,3.139008,12.321263,14.749940,0.75,swiglu,6,16.41855,1.1,500
348,val,299,16.347532,3.049330,10.432255,12.280734,0.75,swiglu,6,16.26320,1.1,500
349,val,249,16.437976,3.054953,12.052590,14.915801,0.75,swiglu,6,16.53491,1.1,500


In [225]:
# load tsp and N=20

def get_pivot_table(env_type, N, epoch=299, enc_layers=6, diff_cut=0.75, index=['epoch', 'ns', 'activation', 'baseline']):
    df = pd.read_csv(f"../result_csv/{env_type}_{N}.csv")
    cond1 = df['epoch'] == epoch
    cond2 = df['enc_layers'] == enc_layers
    cond3 = df['diff_cut'] == diff_cut 
    cond4 = df['diff_cut'] == 1
    df = df[cond1 & cond2 & (cond3 | cond4)]
    pivot_table = df.pivot_table(index=index, values=['score', 'score_std', 'runtime', 'runtime_std'])
    # # reindex the column order as to score mean, score std, runtime mean, runtime std
    pivot_table = pivot_table.reindex(columns=['score', 'score_std', 'runtime', 'runtime_std'])
    return pivot_table

In [226]:
tsp_results = [get_pivot_table('tsp', N) for N in [20, 50, 100]]
all_results_df = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)
all_results_df


20                                   \
                                   score score_std   runtime runtime_std   
epoch ns   activation baseline                                             
299   0    relu       mean      3.849166  0.087340  0.106683    0.000164   
                      val       3.848943  0.084740  0.111188    0.000175   
           swiglu     mean      3.846409  0.084434  0.109879    0.000128   
                      val       3.848185  0.085138  0.112462    0.000178   
      100  relu       mean      3.848645  0.087409  0.098566    0.002793   
                      val       3.848350  0.084755  0.089544    0.001562   
           swiglu     mean      3.846195  0.084505  0.102594    0.004207   
                      val       3.847354  0.085008  0.108760    0.005548   
      250  relu       mean      3.848549  0.087396  0.142930    0.011627   
                      val       3.848350  0.084755  0.117019    0.008553   
           swiglu     mean      3.845970  0.084528  0.156250    0.016239   
                      val       3.847841  0.085045  0.142453    0.014585   
      500  relu       mean      3.849092  0.087306  0.143115    0.026448   
                      val       3.848350  0.084755  0.110884    0.014887   
           swiglu     mean      3.846063  0.084436  0.162155    0.043350   
                      val       3.847841  0.085045  0.168574    0.051304   
      1000 relu       mean      3.849092  0.087306  0.188020    0.081464   
                      val       3.848350  0.084755  0.129800    0.042372   
           swiglu     mean      3.846141  0.084478  0.225597    0.153658   
                      val       3.847841  0.085045  0.235934    0.161515   

                                     50                                   \
                                   score score_std   runtime runtime_std   
epoch ns   activation baseline                                             
299   0    relu       mean      5.736106  0.059724  0.231187    0.000463   
                      val       5.738576  0.063584  0.229614    0.000554   
           swiglu     mean      5.726738  0.069453  0.237449    0.000701   
                      val       5.740503  0.066726  0.233933    0.000357   
      100  relu       mean      5.737484  0.059985  0.372957    0.038249   
                      val       5.731046  0.065711  0.363083    0.039500   
           swiglu     mean      5.724850  0.068898  0.324971    0.032071   
                      val       5.737676  0.066269  0.356330    0.035140   
      250  relu       mean      5.735231  0.059618  1.027504    0.453966   
                      val       5.733597  0.065631  0.885038    0.448591   
           swiglu     mean      5.722453  0.067997  0.756995    0.343890   
                      val       5.739178  0.066753  0.948490    0.359832   
      500  relu       mean      5.734466  0.059488  1.187038    0.852000   
                      val       5.729078  0.065781  1.051615    0.785520   
           swiglu     mean      5.725983  0.069559  0.892633    0.612543   
                      val       5.737743  0.066359  1.108870    0.783436   
      1000 relu       mean      5.733857  0.059513  1.954974    2.788646   
                      val       5.730098  0.064709  1.772289    2.907688   
           swiglu     mean      5.725730  0.069605  1.506560    2.072723   
                      val       5.737280  0.066556  1.817081    2.383270   

                                     100                                   
                                   score score_std    runtime runtime_std  
epoch ns   activation baseline                                             
299   0    relu       mean      7.985253  0.078364   0.449912    0.000449  
                      val       8.048133  0.079650   0.450653    0.000271  
           swiglu     mean      7.956173  0.073098   0.454370    0.000451  
                      val       7.955108  0.072696   0.456002    0.000474  
      1

In [227]:
cvrp_results = [get_pivot_table('cvrp', N) for N in [20, 50, 100]]
all_results_df = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)
all_results_df


20                                   \
                                   score score_std   runtime runtime_std   
epoch ns   activation baseline                                             
299   0    relu       mean      6.409700  0.752305  0.130453    0.000305   
                      val       6.455306  0.798060  0.131891    0.000288   
           swiglu     mean      6.423070  0.729668  0.132642    0.000298   
                      val       6.420135  0.723813  0.132299    0.000250   
      100  relu       mean      6.400961  0.759483  0.357921    0.068225   
                      val       6.445238  0.830283  0.438886    0.086366   
           swiglu     mean      6.422838  0.748686  0.320051    0.047316   
                      val       6.402041  0.743564  0.398179    0.061068   
      250  relu       mean      6.405958  0.754172  0.816664    0.401322   
                      val       6.431462  0.811592  0.961626    0.469751   
           swiglu     mean      6.418946  0.731661  0.735687    0.222004   
                      val       6.394133  0.751608  0.873171    0.313853   
      500  relu       mean      6.406541  0.754331  0.947781    0.693277   
                      val       6.444920  0.835227  1.147389    0.793489   
           swiglu     mean      6.423026  0.734522  0.820323    0.402233   
                      val       6.405495  0.730563  0.982952    0.546372   
      1000 relu       mean      6.407326  0.755713  1.513717    2.072752   
                      val       6.447868  0.804754  1.829762    2.193721   
           swiglu     mean      6.429175  0.743047  1.268608    1.079432   
                      val       6.410132  0.724938  1.542245    1.544175   

                                      50                                   \
                                    score score_std   runtime runtime_std   
epoch ns   activation baseline                                              
299   0    relu       mean      10.804991  1.650085  0.278725    0.001113   
                      val       10.863422  1.648301  0.275667    0.001089   
           swiglu     mean      10.793945  1.696612  0.278183    0.001341   
                      val       10.840164  1.483185  0.277188    0.000455   
      100  relu       mean      10.822530  1.588629  0.965002    0.147288   
                      val       10.871769  1.607463  0.934099    0.161764   
           swiglu     mean      10.774742  1.712863  0.921308    0.196363   
                      val       10.841976  1.527178  1.059928    0.186781   
      250  relu       mean      10.822807  1.614516  2.046018    0.957859   
                      val       10.885779  1.620575  2.090382    1.016583   
           swiglu     mean      10.799106  1.749801  2.007733    1.140759   
                      val       10.845518  1.489249  2.271400    1.310936   
      500  relu       mean      10.819180  1.602545  3.577393    3.442226   
                      val       10.865096  1.645179  3.685498    3.463704   
           swiglu     mean      10.787783  1.721432  3.561608    4.306962   
                      val       10.833599  1.486477  4.015292    4.371571   
      1000 relu       mean      10.808679  1.630902  6.211236   11.211024   
                      val       10.877391  1.671386  6.657273   14.156783   
           swiglu     mean      10.784687  1.669842  6.185035   14.437602   
                      val       10.845643  1.530818  7.069639   15.202203   

                                      100                                   
                                    score score_std    runtime runtime_std  
epoch ns   activation baseline                                              
299   0    relu       mean      16.441851  2.987836   0.523004    0.001074  
                      val       16.446298  2.998878   0.524640    0.000900  
           swiglu     mean      16.404301  3.089149   0.527438    0.001732  
                      val       16.357462  3.040061   

In [231]:
tsp_results = [get_pivot_table('tsp', N, index=['activation']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['activation']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                                         50   \
                    score score_std   runtime runtime_std      score   
     activation                                                        
tsp  relu        3.848689  0.086052  0.123775    0.019004   5.733954   
     swiglu      3.846984  0.084766  0.152466    0.045071   5.731813   
cvrp relu        6.425528  0.785592  0.827609    0.677950  10.844164   
     swiglu      6.414899  0.736207  0.720616    0.421700  10.814716   

                                                       100            \
                score_std   runtime runtime_std      score score_std   
     activation                                                        
tsp  relu        0.062374  0.907530    0.831518   8.018952  0.080887   
     swiglu      0.067817  0.818331    0.662396   7.955087  0.072591   
cvrp relu        1.627958  2.672129    3.455943  16.455584  3.024656   
     swiglu      1.606745  2.764731    4.115497  16.389521  3.063760   

                                        
                   runtime runtime_std  
     activation                         
tsp  relu         4.224810    6.960918  
     swiglu       3.881115    6.798483  
cvrp relu        11.354269   16.348030  
     swiglu       9.526938   16.503769

In [232]:
tsp_results = [get_pivot_table('tsp', N, index=['baseline']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['baseline']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                                         50             \
                  score score_std   runtime runtime_std      score score_std   
     baseline                                                                  
tsp  mean      3.847532  0.085914  0.143579    0.034008   5.730290  0.064384   
     val       3.848140  0.084904  0.132662    0.030068   5.735478  0.065808   
cvrp mean      6.414754  0.746359  0.704385    0.498716  10.801845  1.663723   
     val       6.425673  0.775440  0.843840    0.600933  10.857036  1.570981   

                                           100                       \
                runtime runtime_std      score score_std    runtime   
     baseline                                                         
tsp  mean      0.849227    0.719525   7.968395  0.076000   3.837999   
     val       0.876634    0.774389   8.005644  0.077479   4.267926   
cvrp mean      2.603224    3.584254  16.435670  3.053555  10.466180   
     val       2.833637    3.987187  16.409435  3.034862  10.415026   

                           
              runtime_std  
     baseline              
tsp  mean        7.168259  
     val         6.591142  
cvrp mean       18.769996  
     val        14.081803

In [233]:
tsp_results = [get_pivot_table('tsp', N, index=['activation', 'baseline']) for N in [20, 50, 100]]
tsp_results = pd.concat(tsp_results, keys=[20, 50, 100],  axis=1)

cvrp_results = [get_pivot_table('cvrp', N, index=['activation', 'baseline']) for N in [20, 50, 100]]
cvrp_results = pd.concat(cvrp_results, keys=[20, 50, 100],  axis=1)

all_results_df = pd.concat([tsp_results, cvrp_results], keys=['tsp', 'cvrp'], axis=0)
all_results_df

20                                         50   \
                             score score_std   runtime runtime_std      score   
     activation baseline                                                        
tsp  relu       mean      3.848909  0.087351  0.135863    0.024499   5.735429   
                val       3.848468  0.084752  0.111687    0.013510   5.732479   
     swiglu     mean      3.846156  0.084476  0.151295    0.043516   5.725151   
                val       3.847812  0.085056  0.153637    0.046626   5.738476   
cvrp relu       mean      6.406097  0.755201  0.753307    0.647176  10.815637   
                val       6.444959  0.815983  0.901911    0.708723  10.872692   
     swiglu     mean      6.423411  0.737517  0.655462    0.350257  10.788053   
                val       6.406387  0.734897  0.785769    0.493144  10.841380   

                                                                100            \
                         score_std   runtime runtime_std      score score_std   
     activation baseline                                                        
tsp  relu       mean      0.059666  0.954732    0.826665   7.983006  0.078750   
                val       0.065083  0.860328    0.836371   8.054898  0.083024   
     swiglu     mean      0.069102  0.743721    0.612386   7.953784  0.073250   
                val       0.066533  0.892941    0.712407   7.956390  0.071933   
cvrp relu       mean      1.617335  2.615675    3.151902  16.455791  3.008325   
                val       1.638581  2.728584    3.759984  16.455378  3.040988   
     swiglu     mean      1.710110  2.590773    4.016606  16.415550  3.098785   
                val       1.503381  2.938689    4.214389  16.363492  3.028735   

                                                 
                            runtime runtime_std  
     activation baseline                         
tsp  relu       mean       4.136831    7.754827  
                val        4.312788    6.167008  
     swiglu     mean       3.539167    6.581690  
                val        4.223063    7.015277  
cvrp relu       mean      11.087819   18.449787  
                val       11.620718   14.246272  
     swiglu     mean       9.844541   19.090205  
                val        9.209334   13.917334